# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем необходимые нам библиотеки

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier

Прочитаем датафрейм

In [18]:
df_users = pd.read_csv('/datasets/users_behavior.csv')

Получим первые 50 строк таблицы

In [19]:
df_users.head(50)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Вывод: Наблюдаем из 50 первых строчек только 19 записей с тарифом "Ультра", то есть архивных тарифов 31, что достаточно много.

## Разбейте данные на выборки

Разобьём данные на три выборки - тренировочную и валидационную и тестовую, предварительно разбив валидационную выборку ещё раз на валидационную и тестовую в соотношении 60-20-20

In [20]:
df_users_train, df_users_valid = train_test_split(df_users, test_size=0.40, random_state=12345)
df_users_valid,df_users_test = train_test_split(df_users_valid,test_size = 0.50,random_state = 12345)


In [21]:
len(df_users_train) / (len(df_users_train) + len(df_users_valid) + len(df_users_test))

0.5998755444928439

In [22]:
len(df_users_valid) / (len(df_users_train) + len(df_users_valid) + len(df_users_test))

0.2000622277535781

In [23]:
len(df_users_test) / (len(df_users_train) + len(df_users_valid) + len(df_users_test))

0.2000622277535781

## Исследуйте модели

Создадим переменные для признаков и целевого признака.
Целевой признак - "is_ultra"

In [24]:
features_test = df_users_test.drop(['is_ultra'], axis=1)
target_test = df_users_test['is_ultra'] 
features_train = df_users_train.drop(['is_ultra'], axis=1)
target_train = df_users_train['is_ultra']
features_valid = df_users_valid.drop(['is_ultra'], axis=1)
target_valid = df_users_valid['is_ultra']

Посчитаем accuracy в модели "случайного леса"

In [25]:
best_forest_model = None
best_forest_result = 0
for est in range(1, 11):
    
    forest_model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    forest_model.fit(features_train, target_train)
    forest_result = forest_model.score(features_valid, target_valid) 
    if forest_result > best_forest_result:
        best_forest_model = forest_model
        best_forest_result = forest_result

print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


Посчитаем accuracy на валидационной выборке для линейной регрессии

In [26]:
linear_model =  LinearRegression()
linear_model.fit(features_train,target_train) 
predictions_valid = linear_model.predict(features_valid) 

linear_result = linear_model.score(features_valid, target_valid)
print("Accuracy модели линейной регрессии на валидационной выборке:", linear_result)

Accuracy модели линейной регрессии на валидационной выборке: 0.06970225670616292


Посчитаем accuracy наилучшей модели

In [27]:
best_forest_decisions_model = None
best_forest_decisions_result = 0
for est in range(1, 11):
    
    forest_decisions_model = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth = 40,criterion = 'entropy') 
    forest_decisions_model.fit(features_train, target_train)
    forest_decisions_result = forest_decisions_model.score(features_valid, target_valid) 
    if forest_decisions_result > best_forest_decisions_result:
        best_forest_decisions_model = forest_decisions_model
        best_forest_decisions_result = forest_decisions_result
        
print("Accuracy наилучшей модели на валидационной выборке:", best_forest_decisions_result)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


Добавил ещё 2 признака

## Проверьте модель на тестовой выборке

Проверим модель на тестовой выборке "случайного леса"

In [28]:
best_forest_model_test = None
best_forest_result_test = 0
for est in range(1, 11):
    
    forest_model_test = RandomForestClassifier(random_state=12345, n_estimators=est) 
    forest_model_test.fit(features_train, target_train)
    forest_result_test = forest_model_test.score(features_test, target_test) 
    if forest_result_test > best_forest_result_test:
        best_forest_model_test = forest_model_test
        best_forest_result_test = forest_result_test

print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result_test)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


##  Проверьте модели на адекватность

Для проверки на вменяемость возьмём и проверим со случайной моделью DummyClassifier.

In [31]:
dummy_clf = DummyClassifier(strategy="most_frequent",random_state = 12345)
dummy_clf.fit(features_train, target_train)
print("Accuracy dummy-модели на тестовой выборке:",dummy_clf.score(features_test, target_test))
print("Accuracy нашей модели на тестовой выборке:",forest_model_test.score(features_test, target_test))

Accuracy dummy-модели на тестовой выборке: 0.6842923794712286
Accuracy нашей модели на тестовой выборке: 0.7807153965785381


Вывод:Наша модель более вменяема и предсказывает лучше.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
